# OKM Model #

## Set-up ##

### Imports ###

In [1]:
import pandas as pd
import numpy as np

### Objects ###

In [2]:
class recipe:
    """ a recipe """

    def __init__(self, name : str, id : str, data : pd.DataFrame) -> None:
        """ initialise an instance of recipe"""
        self.name = name
        self.id = id
        self.data = data

    def __str__(self) -> str:
        """ set the string representation of a recipe """
        return f'{self.id} {self.name}'

## Data preparation ##

### Data loading ###

#### BOM ####


In [3]:
bom_data_raw = pd.read_excel("recepten.xlsx", skiprows=1, header=None, decimal=",")

#### Prices & weights ####

In [13]:
price_weight_data = pd.read_excel("Input Price List + Grammage.xlsx", sheet_name="PriceList", header=0).astype({'INGREDIENT CODE': 'string'})

#### Waste ####

In [5]:
waste_data = pd.read_excel("Input Waste Table.xlsx", sheet_name='WASTE', header=0).astype({'MEAL CODE': "string", 'INGREDIENT CODE': 'string'})

##### Add unique id column #####

In [6]:
waste_data['id'] = waste_data[['MEAL CODE', 'INGREDIENT CODE']].agg('_'.join, axis=1).astype('string')

#### Product master ####

In [7]:
product_data = pd.read_excel("Input Productmaster.xlsx", sheet_name='Product')

##### Active recipes #####

In [8]:
active_rec_data = pd.read_excel("Input Productmaster.xlsx", sheet_name='Actief')

In [9]:
active_recipes = [str(x) for x in active_rec_data[active_rec_data['Actief'] == 'Ja']['Artikel']]

##### Split product data by categorie #####
Store the ids ('hf_nr') in NumPy arrays for easy and fast checking against later.

In [10]:
product_data_ingredient = np.array(product_data[product_data['Categorie'] == 'Ingredient']['Nummer'])
product_data_packaging = np.array(product_data[product_data['Categorie'] == 'Verpakking']['Nummer'])
product_data_HF = np.array(product_data[product_data['Categorie'] == 'Halffabrikaat']['Nummer'])
product_data_gas = np.array(product_data[product_data['Categorie'] == 'Gas']['Nummer'])

### Data cleaning ###

#### BOM ####

##### Split data into recipes #####

In [11]:
recipes = []

for i in range(len(bom_data_raw)):
    # a new recipe starts
    if bom_data_raw[4][i] == 'Omschrijving':
        start_idx = i + 1
        recipe_name = bom_data_raw[4][i + 1]
        recipe_id = bom_data_raw[3][i + 1]

        # the recipe ends
        for j in range(i, len(bom_data_raw)):
            if bom_data_raw[3][j] == 'Kostenaandeel voor dit artikel':
                end_idx = j
                recipe_data = bom_data_raw.iloc[(i + 2):j].drop(range(8, 13), axis='columns').reset_index()
                recipe_data = recipe_data.rename(columns={0: "id_nr", 1: "nr", 2: "Niveau", 3: "hf_nr", 4: "Omschrijving", 5: "Aantal (Basis)", 6: "Basiseenheid", 7: "Materiaalkosten"})
                recipe_data = recipe_data.astype({"id_nr": str, "nr": int, "Niveau": int, "hf_nr": str, "Omschrijving": str, "Aantal (Basis)": float, "Basiseenheid": str, "Materiaalkosten": float})
                recipe_data.insert(loc=2, column="Product Naam", value=[recipe_name for i in range(len(recipe_data))])
                recipes.append(recipe(name=recipe_name, id=recipe_id, data=recipe_data))
                i += j
                break

##### Drop inactive recipes #####
Keep this separate for now in case it has to be removed later.

In [12]:
recipes_temp = []
for recipe in recipes:
    if str(recipe.id) in active_recipes:
        recipes_temp.append(recipe)

recipes = recipes_temp 

## Modeling ##

### New prices for ingredients & packaging ###

In [31]:
for recipe in recipes:
    new_prices = []

    for i in range(len(recipe.data)):
        item_id = recipe.data['hf_nr'][i]

        if (item_id in product_data_ingredient) or (item_id in product_data_gas): # ingredients & gas
            subset_price_df = price_weight_data[price_weight_data['INGREDIENT CODE'] == item_id]
            new_price = subset_price_df['PRICE Q1'].iloc[0]
        
        elif item_id in product_data_packaging: # packaging
            new_price = 0

        else: # other - HFs and not found ingredients
            new_price = None
    
        new_prices.append(new_price)
        
    recipe.data['Nieuwe prijs'] = new_prices